In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open('models/many-to-one/adv_trained_models/adv_training.json', 'r') as f:
    data = json.loads(f.read())
    
with open('models/many-to-one/model_info.json', 'r') as f:
    originals = json.loads(f.read())

    
metrics = [
    "R2",
    "RMSE",
    "MAE",
    "MAPE",
]

In [3]:
originals

{'Electricity Transformer Data - 15 min': {'Single Layer LSTM': {'R2': 0.9908708403710925,
   'MAE': 0.26,
   'RMSE': 0.35,
   'MSE': 0.12,
   'MAPE': 5.0,
   'SMAPE': 10.02,
   'MDAPE': 3.09,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--Single Layer LSTM.h5'},
  'Double Layer LSTM': {'R2': 0.9917433226778942,
   'MAE': 0.24,
   'RMSE': 0.33,
   'MSE': 0.11,
   'MAPE': 4.43,
   'SMAPE': 9.45,
   'MDAPE': 2.89,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--Double Layer LSTM.h5'},
  'Bidirectional LSTM': {'R2': 0.9919275919362605,
   'MAE': 0.24,
   'RMSE': 0.33,
   'MSE': 0.11,
   'MAPE': 4.58,
   'SMAPE': 9.63,
   'MDAPE': 2.83,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--Bidirectional LSTM.h5'},
  'CNN1D': {'R2': 0.9800732481560549,
   'MAE': 0.37,
   'RMSE': 0.52,
   'MSE': 0.27,
   'MAPE': 6.19,
   'SMAPE': 13.42,
   'MDAPE': 4.15,
   'path': 'models/many-to-one/Electricity Transformer Data - 15 min--CNN1D.h5

In [4]:
rows = []
for dataset in data.keys():
    for model_name in data[dataset].keys():
        for metric in metrics:
            rows.append((dataset,model_name,metric))
            

columns = [
    ('Original',''), \
    ('FGSM', 'Attack'), ('FGSM', 'Training'), ('FGSM', 'Test'), \
    ('PGD', 'Attack'), ('PGD', 'Training'), ('PGD', 'Test') \
]

row_list = pd.MultiIndex.from_tuples(rows)
col_list = pd.MultiIndex.from_tuples(columns)

A = pd.DataFrame(index = row_list, columns = col_list)

In [5]:
for dataset in originals.keys():
    for model_name in originals[dataset].keys():
        for metric in metrics:
                    A.loc[(dataset, model_name, metric)][('Original', '')] = round(originals[dataset][model_name][metric] ,2)           

In [6]:
for dataset in data.keys():
    for model_name in data[dataset].keys():
        for adv_attack in data[dataset][model_name].keys():
            for setting in data[dataset][model_name][adv_attack].keys():
                if setting == "path": continue
                if "test" in setting:
                    parsed_setting = "Test"
                if "train" in setting:
                    parsed_setting = "Training"
                for metric in metrics:
                    A.loc[(dataset, model_name, metric)][(adv_attack, parsed_setting)] = data[dataset][model_name][adv_attack][setting][metric]

In [7]:
with open("adv_examples/many-to-one/adv_gen_l_inf_2.json") as f:
    attack = json.loads(f.read())
    
for dataset in attack.keys():
    for model_name in attack[dataset].keys():
        for adv_attack in attack[dataset][model_name].keys():
            
            if adv_attack == "FGSM":
                parsed_setting = "epsilon=0.025"
            elif adv_attack == "PGD":
                parsed_setting = "alpha=0.025 | epsilon=0.025"
                
            for metric in metrics:
                A.loc[(dataset, model_name, metric)][(adv_attack, "Attack")] = attack[dataset][model_name][adv_attack][parsed_setting]["metrics"][metric]                


In [8]:
A.head(50)

Original  \
                                                                             
Electricity Transformer Data - 15 min Single Layer LSTM  R2           0.99   
                                                         RMSE         0.35   
                                                         MAE          0.26   
                                                         MAPE            5   
                                      Double Layer LSTM  R2           0.99   
                                                         RMSE         0.33   
                                                         MAE          0.24   
                                                         MAPE         4.43   
                                      Bidirectional LSTM R2           0.99   
                                                         RMSE         0.33   
                                                         MAE          0.24   
                                                         MAPE         4.58   
                                      CNN1D              R2           0.98   
                                                         RMSE         0.52   
                                                         MAE          0.37   
                                                         MAPE         6.19   
Metro Interstate Human Traffic Volume Single Layer LSTM  R2          -1.24   
                                                         RMSE  2.15141e+07   
                                                         MAE   1.74895e+07   
                                                         MAPE        68.82   
                                      Double Layer LSTM  R2          -1.05   
                                                         RMSE  2.05644e+07   
                                                         MAE   1.71565e+07   
                                                         MAPE         88.6   
                                      Bidirectional LSTM R2          -0.67   
                                                         RMSE  1.85816e+07   
                                                         MAE    1.5102e+07   
                                                         MAPE        71.21   
                                      CNN1D              R2           0.95   
                                                         RMSE   3.3696e+06   
                                                         MAE   2.37484e+06   
                                                         MAPE        18.65   
Beijing-Guanyuan Air-Quality          Single Layer LSTM  R2           0.95   
                                                         RMSE        19.64   
                                                         MAE         10.85   
                                                         MAPE        35.76   
                                      Double Layer LSTM  R2           0.95   
                                                         RMSE        20.35   
                                                         MAE         11.73   
                                                         MAPE        28.05   
                                      Bidirectional LSTM R2           0.95   
                                                         RMSE        19.47   
                                                         MAE         10.55   
                                                         MAPE        30.08   
                                      CNN1D              R2           0.93   
                                                         RMSE        22.96   
                                                         MAE         14.19   
                                                         MAPE        51.83   
Solar Generation - EnerjiSA           Single Layer LSTM  R2           0.99   
                                                         RMSE        12.14   

         

In [9]:
A.to_csv('adversarial_training_results.csv')